In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from patsy import dmatrices

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [2]:
# pbp_data = pd.read_csv("./results/base-epa.csv", encoding="utf-8")
pbp_data = pd.DataFrame()
years = range(2014,2020)
for i in years:
    url = f"https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_{i}.parquet"
    print(f"Pulling url {url}")
    i_data = pd.read_parquet(url, engine='pyarrow')

    #sort=True eliminates a warning and alphabetically sorts columns
    pbp_data = pbp_data.append(i_data, sort=True)

#Give each row a unique index
pbp_data.reset_index(drop=True, inplace=True)
pbp_data.head()

Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2014.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2015.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2016.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2017.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2018.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2019.parquet


,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [3]:
game_data = pd.DataFrame()
spread_data = pd.DataFrame()

# downloaded files from collegefootballdata.com
def retrieveCfbDataFile(endpoint, year):
    return pd.read_csv(f"data/{endpoint}/{year}.csv", encoding='latin-1')

for i in range(2014, 2020):
    gm = retrieveCfbDataFile('games',i)
    gm['game_id'] = gm['id']
    gm['year'] = i
    gm['winner'] = np.where(gm.home_points > gm.away_points, gm.home_team, gm.away_team)
    game_data = game_data.append(gm, sort=False)
    
    bets = retrieveCfbDataFile('lines', i)
    bets = bets[bets.lineProvider == "consensus"]
    bets['game_id'] = bets['id']
    bets['year'] = i
    bets['spread'] = -1 * bets.spread # spread is negative for home favorite; positive for away favorite, but NFLfastR uses "(how many points home team was favored by) for this"
    spread_data = spread_data.append(bets, sort=False)
    

print(f"Total Games: {len(game_data)}")
print(f"Total Spreads: {len(spread_data)}")

Total Games: 5018
Total Spreads: 4775


In [4]:
pbp_data = pd.merge(pbp_data, game_data[['game_id','winner']], left_on="game_id", right_on="game_id")
pbp_data = pbp_data[pbp_data.game_id != 400603838]
pbp_data.head()

,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [5]:
pbp_data = pd.merge(pbp_data, spread_data[['game_id','spread']], left_on="game_id", right_on="game_id")
pbp_data.head()

,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [6]:
pbp_data['pos_team_spread'] = np.where(pbp_data.pos_team == pbp_data.home, pbp_data.spread, -1 * pbp_data.spread)
pbp_data['elapsed_share'] = (3600 - pbp_data.adj_TimeSecsRem) / 3600
pbp_data['spread_time'] = pbp_data.pos_team_spread * np.exp(-4 * pbp_data.elapsed_share)
pbp_data['is_home'] = pbp_data.pos_team == pbp_data.home
pbp_data.head()

,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [7]:
pbp_data['win_indicator'] = pbp_data.pos_team == pbp_data.winner
pbp_data.half = pbp_data.half.astype(int)
pbp_data.loc[pbp_data.off_timeouts_rem_before.isna(), "off_timeouts_rem_before"] = 0
pbp_data.loc[pbp_data.def_timeouts_rem_before.isna(), "def_timeouts_rem_before"] = 0

# https://zacharyst.com/2019/03/06/group_by-mutate-using-pandas/
# def determine_receive_2h_ko(data):
# #     print(data.game_id.iloc[0])
#     val = data.sort_values(by="play_id")[(data.period == 3) & (~data.play_type.str.contains("Kickoff"))].pos_team
#     ret_val = None
#     if (len(val) != 0):
#         ret_val = val.iloc[0]
#     data['receive_2h_ko'] = ret_val
#     return data

# pbp_data = pbp_data.groupby('game_id').apply(determine_receive_2h_ko)
# pbp_data.loc[pbp_data.turnover_vec_lag.isna(), 'turnover_vec_lag'] = 0
pbp_data.head()

,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [8]:
# test = pbp_data[pbp_data.game_id == pbp_data.game_id.iloc[0]].sort_values(by="play_id")
# test[(test.period == 3) & (~test.play_type.str.contains("Kickoff"))].pos_team.iloc[0]


In [9]:
nrounds = 760
params = {
    "booster":"gbtree",
    "objective":"binary:logistic",
    "eval_metric": ["logloss"],
    "eta":0.02,
    "gamma":0.3445502,
    "subsample":0.7204741,
    "colsample_bytree":0.5714286,
    "max_depth":5,
    "min_child_weight":14
}

filtered_pbp_data = pbp_data[
    (pbp_data.ep_before.notna())
    & (pbp_data.score_diff.notna())
    & (pbp_data.play_type.notna())
    & (pbp_data.win_indicator.notna())
    & (pbp_data.off_timeouts_rem_before.notna())
    & (pbp_data.def_timeouts_rem_before.notna())
    & (pbp_data.yards_to_goal.notna())
    & (pbp_data.period <= 4)
]

dtrain = xgb.DMatrix(filtered_pbp_data[["pos_team_receives_2H_kickoff","spread_time","TimeSecsRem","adj_TimeSecsRem","ExpScoreDiff_Time_Ratio","pos_score_diff_start","down","distance","yards_to_goal","is_home","pos_team_timeouts_rem_before","def_pos_team_timeouts_rem_before","period"]], label=filtered_pbp_data.win_indicator)
dtrain

In [10]:
wp_model = xgb.train(params, dtrain, num_boost_round=nrounds, verbose_eval=2)
# save for debug
wp_model.dump_model('wp_xgb_dump.json', with_stats=True, dump_format='json')


In [11]:
# Test with 2020 data
base_data = pd.read_parquet("https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2020.parquet")
# print(base_data.columns.to_list())

games_2020 = retrieveCfbDataFile('games', 2020)
games_2020['game_id'] = games_2020['ï»¿id']
games_2020['year'] = 2020
games_2020 = games_2020[games_2020.home_points != games_2020.away_points]
games_2020['winner'] = np.where(games_2020.home_points > games_2020.away_points, games_2020.home_team, games_2020.away_team)

spread_2020 = retrieveCfbDataFile('lines', 2020)
spread_2020 = spread_2020[spread_2020.lineProvider == "consensus"]
spread_2020['spread'] = -1 * spread_2020.spread # spread is negative for home favorite; positive for away favorite, but NFLfastR uses "(how many points home team was favored by) for this"
spread_2020['game_id'] = spread_2020['ï»¿id']
spread_2020['year'] = 2020
base_data = pd.merge(base_data, spread_2020[['game_id','spread']], left_on="game_id", right_on="game_id")
base_data = pd.merge(base_data, games_2020[['game_id','winner']], left_on="game_id", right_on="game_id")
base_data.head()

,year,week,id_play,game_id,game_play_number,half_play_number,drive_play_number,pos_team,def_pos_team,pos_team_score,def_pos_team_score,half,period,clock.minutes,clock.seconds,play_type,play_text,down,distance,yards_to_goal,yards_gained,EPA,ep_before,ep_after,wpa,wp_before,wp_after,def_wp_before,def_wp_after,penalty_detail,yds_penalty,penalty_1st_conv,new_series,firstD_by_kickoff,firstD_by_poss,firstD_by_penalty,firstD_by_yards,def_EPA,home_EPA,away_EPA,home_EPA_rush,away_EPA_rush,home_EPA_pass,away_EPA_pass,total_home_EPA,total_away_EPA,total_home_EPA_rush,total_away_EPA_rush,total_home_EPA_pass,total_away_EPA_pass,net_home_EPA,net_away_EPA,net_home_EPA_rush,net_away_EPA_rush,net_home_EPA_pass,net_away_EPA_pass,success,epa_success,rz_play,scoring_opp,middle_8,stuffed_run,change_of_pos_team,downs_turnover,turnover,pos_score_diff_start,pos_score_pts,log_ydstogo,ExpScoreDiff,ExpScoreDiff_Time_Ratio,half_clock.minutes,TimeSecsRem,adj_TimeSecsRem,Goal_To_Go,Under_two,home,away,home_wp_before,away_wp_before,home_wp_after,away_wp_after,end_of_half,pos_team_receives_2H_kickoff,lead_pos_team,lead_play_type,lag_pos_team,lag_play_type,orig_play_type,Under_three,down_end,distance_end,log_ydstogo_end,yards_to_goal_end,TimeSecsRem_end,Goal_To_Go_end,Under_two_end,offense_score_play,defense_score_play,ppa,yard_line,scoring,pos_team_timeouts_rem_before,def_pos_team_timeouts_rem_before,pos_team_timeouts,def_pos_team_timeouts,pos_score_diff,pos_score_diff_start_end,offense_play,defense_play,offense_receives_2H_kickoff,change_of_poss,score_pts,score_diff_start,score_diff,offense_score,defense_score,offense_conference,defense_conference,off_timeout_called,def_timeout_called,offense_timeouts,defense_timeouts,off_timeouts_rem_before,def_timeouts_rem_before,rusher_player_name,yds_rushed,passer_player_name,receiver_player_name,yds_receiving,yds_sacked,sack_players,sack_player_name,sack_player_name2,pass_breakup_player_name,interception_player_name,yds_int_return,fumble_player_name,fumble_forced_player_name,fumble_recovered_player_name,yds_fumble_return,punter_player_name,yds_punted,punt_returner_player_name,yds_punt_return,yds_punt_gained,punt_block_player_name,punt_block_return_player_name,fg_kicker_player_name,yds_fg,fg_block_player_name,fg_return_player_name,kickoff_player_name,yds_kickoff,kickoff_returner_player_name,yds_kickoff_return,new_id,orig_drive_number,drive_number,drive_result_detailed,new_drive_pts,drive_id,drive_result,drive_start_yards_to_goal,drive_end_yards_to_goal,drive_yards,drive_scoring,drive_pts,drive_start_period,drive_end_period,drive_time_minutes_start,drive_time_seconds_start,drive_time_minutes_end,drive_time_seconds_end,drive_time_minutes_elapsed,drive_time_seconds_elapsed,drive_numbers,number_of_drives,pts_scored,drive_result_detailed_flag,drive_result2,drive_num,lag_drive_result_detailed,lead_drive_result_detailed,lag_new_drive_pts,id_drive,rush,rush_td,pass,pass_td,completion,pass_attempt,target,sack_vec,sack,int,int_td,turnover_vec,turnover_vec_lag,turnover_indicator,kickoff_play,receives_2H_kickoff,missing_yard_flag,scoring_play,td_play,touchdown,safety,fumble_vec,kickoff_tb,kickoff_onside,kickoff_oob,kickoff_fair_catch,kickoff_downed,kickoff_safety,kick_play,punt,punt_play,punt_tb,punt_oob,punt_fair_catch,punt_downed,punt_safety,punt_blocked,penalty_safety,fg_inds,fg_made,fg_make_prob,No_Score_before,FG_before,Opp_FG_before,Opp_Safety_before,Opp_TD_before,Safety_before,TD_before,No_Score_after,FG_after,Opp_FG_after,Opp_Safety_after,Opp_TD_after,Safety_after,TD_after,position_reception,position_target,position_completion,position_incompletion,position_sack_taken,position_sack,position_interception_thrown,position_interception,position_fumble,position_fumble_forced,position_fumble_recovered,position_pass_breakup,position_rush,position_touchdown,season,opponent,team_score,opponent_score,rush_player_id,rush_player,rush_yds,reception_player_id,reception_player,reception_yds,completion_player_id,completion_player,completio

In [12]:
base_data[base_data.down_end.isna()].play_type.value_counts()

End Period                       1038
End of Half                       434
End of Game                       129
Rush                              110
Pass Incompletion                  39
Pass Reception                     31
Timeout                            20
Passing Touchdown                  15
Rushing Touchdown                  14
Penalty                            14
Field Goal Good                     9
Sack                                8
Field Goal Missed                   6
Interception Return                 5
End of Regulation                   3
Fumble Recovery (Opponent)          2
Blocked Field Goal                  2
Interception Return Touchdown       2
Name: play_type, dtype: int64

In [13]:
base_data['win_indicator'] = base_data.pos_team == base_data.winner
base_data['ExpScoreDiff_end'] = base_data.score_diff_start + base_data.ep_after
base_data['adj_TimeSecsRem_end'] = np.where(base_data.half == 1, 1800 + base_data.adj_TimeSecsRem, base_data.adj_TimeSecsRem)
base_data['ExpScoreDiff_Time_Ratio_end'] = base_data.ExpScoreDiff_end / (base_data.adj_TimeSecsRem_end + 1)
base_data['pos_team_spread'] = np.where(base_data.pos_team == base_data.home, base_data.spread, -1 * base_data.spread)
base_data['elapsed_share'] = (3600 - base_data.adj_TimeSecsRem) / 3600
base_data['elapsed_share_end'] = (3600 - base_data.adj_TimeSecsRem_end) / 3600
base_data['spread_time'] = base_data.pos_team_spread * np.exp(-4 * base_data.elapsed_share)
base_data['spread_time_end'] = base_data.pos_team_spread * np.exp(-4 * base_data.elapsed_share_end)
base_data['is_home'] = base_data.pos_team == base_data.home
base_data.head()


,year,week,id_play,game_id,game_play_number,half_play_number,drive_play_number,pos_team,def_pos_team,pos_team_score,def_pos_team_score,half,period,clock.minutes,clock.seconds,play_type,play_text,down,distance,yards_to_goal,yards_gained,EPA,ep_before,ep_after,wpa,wp_before,wp_after,def_wp_before,def_wp_after,penalty_detail,yds_penalty,penalty_1st_conv,new_series,firstD_by_kickoff,firstD_by_poss,firstD_by_penalty,firstD_by_yards,def_EPA,home_EPA,away_EPA,home_EPA_rush,away_EPA_rush,home_EPA_pass,away_EPA_pass,total_home_EPA,total_away_EPA,total_home_EPA_rush,total_away_EPA_rush,total_home_EPA_pass,total_away_EPA_pass,net_home_EPA,net_away_EPA,net_home_EPA_rush,net_away_EPA_rush,net_home_EPA_pass,net_away_EPA_pass,success,epa_success,rz_play,scoring_opp,middle_8,stuffed_run,change_of_pos_team,downs_turnover,turnover,pos_score_diff_start,pos_score_pts,log_ydstogo,ExpScoreDiff,ExpScoreDiff_Time_Ratio,half_clock.minutes,TimeSecsRem,adj_TimeSecsRem,Goal_To_Go,Under_two,home,away,home_wp_before,away_wp_before,home_wp_after,away_wp_after,end_of_half,pos_team_receives_2H_kickoff,lead_pos_team,lead_play_type,lag_pos_team,lag_play_type,orig_play_type,Under_three,down_end,distance_end,log_ydstogo_end,yards_to_goal_end,TimeSecsRem_end,Goal_To_Go_end,Under_two_end,offense_score_play,defense_score_play,ppa,yard_line,scoring,pos_team_timeouts_rem_before,def_pos_team_timeouts_rem_before,pos_team_timeouts,def_pos_team_timeouts,pos_score_diff,pos_score_diff_start_end,offense_play,defense_play,offense_receives_2H_kickoff,change_of_poss,score_pts,score_diff_start,score_diff,offense_score,defense_score,offense_conference,defense_conference,off_timeout_called,def_timeout_called,offense_timeouts,defense_timeouts,off_timeouts_rem_before,def_timeouts_rem_before,rusher_player_name,yds_rushed,passer_player_name,receiver_player_name,yds_receiving,yds_sacked,sack_players,sack_player_name,sack_player_name2,pass_breakup_player_name,interception_player_name,yds_int_return,fumble_player_name,fumble_forced_player_name,fumble_recovered_player_name,yds_fumble_return,punter_player_name,yds_punted,punt_returner_player_name,yds_punt_return,yds_punt_gained,punt_block_player_name,punt_block_return_player_name,fg_kicker_player_name,yds_fg,fg_block_player_name,fg_return_player_name,kickoff_player_name,yds_kickoff,kickoff_returner_player_name,yds_kickoff_return,new_id,orig_drive_number,drive_number,drive_result_detailed,new_drive_pts,drive_id,drive_result,drive_start_yards_to_goal,drive_end_yards_to_goal,drive_yards,drive_scoring,drive_pts,drive_start_period,drive_end_period,drive_time_minutes_start,drive_time_seconds_start,drive_time_minutes_end,drive_time_seconds_end,drive_time_minutes_elapsed,drive_time_seconds_elapsed,drive_numbers,number_of_drives,pts_scored,drive_result_detailed_flag,drive_result2,drive_num,lag_drive_result_detailed,lead_drive_result_detailed,lag_new_drive_pts,id_drive,rush,rush_td,pass,pass_td,completion,pass_attempt,target,sack_vec,sack,int,int_td,turnover_vec,turnover_vec_lag,turnover_indicator,kickoff_play,receives_2H_kickoff,missing_yard_flag,scoring_play,td_play,touchdown,safety,fumble_vec,kickoff_tb,kickoff_onside,kickoff_oob,kickoff_fair_catch,kickoff_downed,kickoff_safety,kick_play,punt,punt_play,punt_tb,punt_oob,punt_fair_catch,punt_downed,punt_safety,punt_blocked,penalty_safety,fg_inds,fg_made,fg_make_prob,No_Score_before,FG_before,Opp_FG_before,Opp_Safety_before,Opp_TD_before,Safety_before,TD_before,No_Score_after,FG_after,Opp_FG_after,Opp_Safety_after,Opp_TD_after,Safety_after,TD_after,position_reception,position_target,position_completion,position_incompletion,position_sack_taken,position_sack,position_interception_thrown,position_interception,position_fumble,position_fumble_forced,position_fumble_recovered,position_pass_breakup,position_rush,position_touchdown,season,opponent,team_score,opponent_score,rush_player_id,rush_player,rush_yds,reception_player_id,reception_player,reception_yds,completion_player_id,completion_player,completio

In [14]:
# print(len(test_data))
# len(test_data[test_data.win_indicator == 1])

In [15]:
test_data = base_data[
    (base_data.ep_before.notna())
    & (base_data.pos_score_diff_start.notna())
    & (base_data.play_type.notna())
    & (base_data.win_indicator.notna())
    & (base_data.pos_team_timeouts_rem_before.notna())
    & (base_data.def_pos_team_timeouts_rem_before.notna())
    & (base_data.yards_to_goal.notna())
    & (base_data.period <= 4)
    
    & (base_data.ep_after.notna())
    & (base_data.pos_score_diff_start_end.notna())
    & (base_data.pos_team_timeouts.notna())
    & (base_data.def_pos_team_timeouts.notna())
    & (base_data.yards_to_goal_end.notna())
    
    & (base_data.down_end.notna())
]
test_data.down_end = test_data.down_end.astype(int)
dtest_start = xgb.DMatrix(test_data[["pos_team_receives_2H_kickoff","spread_time","TimeSecsRem","adj_TimeSecsRem","ExpScoreDiff_Time_Ratio","pos_score_diff_start","down","distance","yards_to_goal","is_home","pos_team_timeouts_rem_before","def_pos_team_timeouts_rem_before","period"]], label=test_data.win_indicator)

WP_start = wp_model.predict(dtest_start)

/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [16]:
end_data = test_data[["pos_team_receives_2H_kickoff","spread_time_end","TimeSecsRem_end","adj_TimeSecsRem_end","ExpScoreDiff_Time_Ratio_end","pos_score_diff_start_end","down_end","distance_end","yards_to_goal_end","is_home","pos_team_timeouts","def_pos_team_timeouts","period"]]
end_data.columns = ["pos_team_receives_2H_kickoff","spread_time","TimeSecsRem","adj_TimeSecsRem","ExpScoreDiff_Time_Ratio","pos_score_diff_start","down","distance","yards_to_goal","is_home","pos_team_timeouts_rem_before","def_pos_team_timeouts_rem_before","period"]

dtest_end = xgb.DMatrix(end_data, label=test_data.win_indicator)
WP_end = wp_model.predict(dtest_end)

In [17]:
test_data['WP_NFL_start'] = WP_start
test_data['WP_NFL_end'] = WP_end
test_data['WPA_NFL'] = test_data['WP_NFL_end'] - test_data['WP_NFL_start']

print("Diff from cfbscrapR model:")
print(f"Diff of WPA: {(test_data['wpa'] - test_data['WPA_NFL']).mean()}")
print(f"Diff of WP_start: {(test_data['wp_before'] - test_data['WP_NFL_start']).mean()}")
print(f"Diff of WP_end: {(test_data['wp_after'] - test_data['WP_NFL_end']).mean()}")

Diff from cfbscrapR model:
Diff of WPA: 0.0002346794132986934
Diff of WP_start: -0.001948620258797359
Diff of WP_end: -0.0018170338079118256


/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [18]:
test_data[["wp_before","wp_after","wpa","WP_NFL_start","WP_NFL_end","WPA_NFL"]].head()

,wp_before,wp_after,wpa,WP_NFL_start,WP_NFL_end,WPA_NFL
0,0.490320,0.490320,0.000000,0.960585,0.960880,0.000295
1,0.490320,0.452941,-0.037379,0.960880,0.959535,-0.001345
2,0.452941,0.430033,-0.022908,0.959535,0.955923,-0.003612
3,0.430033,0.574210,0.144177,0.955923,0.969029,0.013106
4,0.574210,0.561482,-0.012728,0.969029,0.968305,-0.000724


In [19]:
gathered = test_data.melt(id_vars=["period","win_indicator"], value_vars=["WP_NFL_start"],value_name="pred_prob")
gathered

,period,win_indicator,variable,pred_prob
0,1,True,WP_NFL_start,0.960585
1,1,True,WP_NFL_start,0.960880
2,1,True,WP_NFL_start,0.959535
3,1,True,WP_NFL_start,0.955923
4,1,True,WP_NFL_start,0.969029
...,...,...,...,...
78507,4,False,WP_NFL_start,0.007487
78508,4,False,WP_NFL_start,0.010563
78509,4,False,WP_NFL_start,0.009123
78510,4,False,WP_NFL_start,0.008774


In [20]:
gathered['bin_pred_prob'] = (round(gathered.pred_prob / 0.05) * .05)
gathered

,period,win_indicator,variable,pred_prob,bin_pred_prob
0,1,True,WP_NFL_start,0.960585,0.95
1,1,True,WP_NFL_start,0.960880,0.95
2,1,True,WP_NFL_start,0.959535,0.95
3,1,True,WP_NFL_start,0.955923,0.95
4,1,True,WP_NFL_start,0.969029,0.95
...,...,...,...,...,...
78507,4,False,WP_NFL_start,0.007487,0.00
78508,4,False,WP_NFL_start,0.010563,0.00
78509,4,False,WP_NFL_start,0.009123,0.00
78510,4,False,WP_NFL_start,0.008774,0.00


In [21]:
gathered['bin_pred_prob'].value_counts()

0.00    11078
1.00    10192
0.95     5884
0.05     5877
0.10     4094
0.90     3768
0.85     3367
0.15     3210
0.80     2873
0.20     2732
0.75     2633
0.25     2605
0.70     2559
0.30     2440
0.65     2318
0.40     2292
0.35     2273
0.60     2125
0.45     2090
0.50     2058
0.55     2044
Name: bin_pred_prob, dtype: int64

In [22]:
grouped_score = gathered.groupby(['period','bin_pred_prob'])
def organize(grp):
    d = {}
    d['n_plays'] = len(grp)
    d['n_wins'] = len(grp[grp.win_indicator == True])
    d['bin_actual_prob'] = d['n_wins'] / d['n_plays']
    return pd.Series(d, index=['n_plays','n_wins','bin_actual_prob'])

organized = grouped_score.apply(organize)
organized = organized.reset_index()
organized

,period,bin_pred_prob,n_plays,n_wins,bin_actual_prob
0,1,0.00,1038.0,28.0,0.026975
1,1,0.05,1231.0,25.0,0.020309
2,1,0.10,994.0,95.0,0.095573
3,1,0.15,984.0,129.0,0.131098
4,1,0.20,987.0,212.0,0.214792
5,1,0.25,889.0,232.0,0.260967
6,1,0.30,723.0,246.0,0.340249
7,1,0.35,690.0,235.0,0.340580
8,1,0.40,861.0,303.0,0.351916
9,1,0.45,771.0,349.0,0.452659


In [23]:
organized['cal_diff'] = abs(organized.bin_pred_prob - organized.bin_actual_prob)
organized

,period,bin_pred_prob,n_plays,n_wins,bin_actual_prob,cal_diff
0,1,0.00,1038.0,28.0,0.026975,0.026975
1,1,0.05,1231.0,25.0,0.020309,0.029691
2,1,0.10,994.0,95.0,0.095573,0.004427
3,1,0.15,984.0,129.0,0.131098,0.018902
4,1,0.20,987.0,212.0,0.214792,0.014792
5,1,0.25,889.0,232.0,0.260967,0.010967
6,1,0.30,723.0,246.0,0.340249,0.040249
7,1,0.35,690.0,235.0,0.340580,0.009420
8,1,0.40,861.0,303.0,0.351916,0.048084
9,1,0.45,771.0,349.0,0.452659,0.002659


In [24]:
def organize_cal_err(grp):
    d = {}
    d['weight_cal_err'] = np.average(grp.cal_diff, weights=grp.n_plays)
    d['n_wins'] = sum(grp.n_wins)
    return pd.Series(d, index=['weight_cal_err', 'n_wins'])

cv_cal_error = organized.groupby('period')
final_cal_error = cv_cal_error.apply(organize_cal_err)
final_cal_error = final_cal_error.reset_index()
final_cal_error

,period,weight_cal_err,n_wins
0,1,0.023119,9278.0
1,2,0.025811,10856.0
2,3,0.024609,9053.0
3,4,0.016807,9406.0


In [25]:
weighted_cal_err_final = np.average(final_cal_error.weight_cal_err, weights=final_cal_error.n_wins)
print(f"Weighted calibration error: {weighted_cal_err_final}")

Weighted calibration error: 0.022687541336426535


In [30]:
# # CoreML model
import coremltools as cml
print(cml.version)

cml_inputs = {
    "pos_team_receives_2H_kickoff" : "If the current offense will receive the second half kickoff",
    "spread_time" : "The number of points that the offense is favored by, scaled by e^x, where x is the elapsed portion of the game ((3600 - adj_TimeSecsRem) / 3600)",
    "TimeSecsRem" : "The number of seconds left in the half.",
    "adj_TimeSecsRem" : "The number of seconds left in the game.",
    "ExpScoreDiff_Time_Ratio" : "The expected point differential divided by the number of seconds remaining in the game (plus 1).",
    "pos_score_diff_start" : "The current difference in score from the offense's perspective.",
    "down" : "The current down.",
    "distance" : "How many yards the offense must gain for a first down.",
    "yards_to_goal" : "The yards left until the current offense reaches the goal line.",
    "is_home" : "If the current offense is the home team",
    "pos_team_timeouts_rem_before" : "How many timeouts the current offense has remaining",
    "def_pos_team_timeouts_rem_before" : "How many timeouts the current defense has remaining",
    "period" : "The current quarter."
}


cml_model = cml.converters.xgboost.convert(wp_model, target="win_indicator")

cml_model.author = 'Ben Baldwin (@benbbaldwin on Twitter, @guga31bb on GitHub) and Sebastian Carl (@mrcaseb on Twitter/GitHub) for model creation code; Saiem Gilani and Meyappan Subbiah for cfbscrapR training data; Akshay Easwaran for Python port.'
cml_model.license = 'MIT'
cml_model.short_description = 'Predicts the expected completion percentage of a given pass attempt. Translated from R, original model available as part of https://github.com/mrcaseb/nflfastR/.'

# Set feature descriptions manually
for (column, desc) in cml_inputs.items():
    cml_model.input_description[column] = desc

# Set the output descriptions
cml_model.output_description['win_indicator'] = 'The chance of the pass attempt being completed.'

# Save the model
cml_model.save('cfb_wpa.mlmodel')
cml_model.get_spec()

<module 'coremltools.version' from '/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/coremltools/version.py'>


KeyError: 'split_condition'

In [28]:
test_json = pd.read_json("wp_xgb_dump.json")
test_json.head()

,nodeid,depth,split,split_condition,yes,no,missing,gain,cover,children
0,0,0,pos_score_diff_start,-2.000000,1,2,1.0,209402.078,152255.750,"[{'nodeid': 1, 'depth': 1, 'split': 'pos_score..."
1,0,0,pos_score_diff_start,-2.000000,1,2,1.0,200836.734,152313.969,"[{'nodeid': 1, 'depth': 1, 'split': 'spread_ti..."
2,0,0,ExpScoreDiff_Time_Ratio,0.000109,1,2,1.0,198857.547,152278.500,"[{'nodeid': 1, 'depth': 1, 'split': 'pos_score..."
3,0,0,ExpScoreDiff_Time_Ratio,0.000097,1,2,1.0,191606.734,152217.328,"[{'nodeid': 1, 'depth': 1, 'split': 'spread_ti..."
4,0,0,pos_score_diff_start,0.000000,1,2,1.0,179378.578,151851.984,"[{'nodeid': 1, 'depth': 1, 'split': 'spread_ti..."


In [29]:
len(test_json[test_json['split_condition'].isna()])

37